In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
sys.path.append("..")
from modules.sequential import Sequential
from modules.linear import Linear
from modules.softmax import Softmax
from modules.relu import Relu
from modules.tanh import Tanh
from modules.convolution import Convolution
from modules.avgpool import AvgPool
from modules.maxpool import MaxPool
from modules.utils import Utils, Summaries, plot_relevances
import modules.render as render

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import numpy as np
import pdb
import scipy.io as sio
import numpy as np
import pandas as pd
from pylab import rcParams
import pickle
import copy
import random
import math
import time

flags = tf.flags
logging = tf.logging

rcParams['figure.figsize'] = 8, 10
#128
F1=open("E:/revisiondata/holiday_2_train_2016",'rb')
#100
F2=open("E:/revisiondata/holiday_2_test_2016",'rb')
#50
F3=open("E:/revisiondata/holiday_2_test_2017",'rb')
train=pickle.load(F1)
validation=pickle.load(F2)
test=pickle.load(F3)
F1.close()
F2.close()
F3.close()

flags.DEFINE_integer("max_steps", 30001,'Number of steps to run trainer.')
flags.DEFINE_integer("batch_size", 5,'Number of steps to run trainer.')
flags.DEFINE_integer("test_every", 100,'Number of steps to run trainer.')
flags.DEFINE_float("learning_rate", 1e-5,'Initial learning rate')
flags.DEFINE_float("dropout", 1, 'Keep probability for training dropout.')
flags.DEFINE_boolean("relevance", True,'Compute relevances')
flags.DEFINE_string("relevance_method", 'alphabeta','relevance methods: simple/epsilon/ww/flat/alphabeta')


FLAGS = flags.FLAGS

def nn():
    
    return Sequential([Convolution(weights_init= tf.truncated_normal_initializer(stddev=0.1),keep_prob=FLAGS.dropout,output_depth=50,input_depth=48,batch_size=FLAGS.batch_size,kernel_size=5,input_dim=30, act ='relu', stride_size=1, pad='SAME'),
                       MaxPool(pad='SAME'),
                       #AvgPool(pad='SAME'),
                       
                       Convolution(weights_init= tf.truncated_normal_initializer(stddev=0.1),keep_prob=FLAGS.dropout,kernel_size=4,output_depth=20,stride_size=1,batch_norm = False,batch_norm_params = {'momentum':0.9, 'epsilon':1e-3, 'training':True ,'name':'bn'},act ='relu', pad='SAME'),
                       MaxPool(pad='SAME'),
                       #AvgPool(pad='SAME'),
                       
                       Convolution(weights_init= tf.truncated_normal_initializer(stddev=0.1),keep_prob=FLAGS.dropout,kernel_size=4,output_depth=20,stride_size=1,batch_norm = True,batch_norm_params = {'momentum':0.9, 'epsilon':1e-3, 'training':True ,'name':'bn'},act ='relu', pad='SAME'),
                       MaxPool(pad='SAME'),
                       #AvgPool(pad='SAME'),
                       
                       Linear(weights_init= tf.truncated_normal_initializer(stddev=0.1),input_dim=320,output_dim=100,batch_norm = False, batch_norm_params = {'momentum':0.9, 'epsilon':1e-3, 'training':True,'name':'bn'}),
                       
                       Linear(weights_init= tf.truncated_normal_initializer(stddev=0.1),input_dim=100,output_dim=2, act ='relu'),
                       #Convolution(kernel_size=1, output_depth=12,stride_size=1, pad='VALID'),
                       #Softmax()
    ])




In [ ]:
##Using this model

tf.reset_default_graph()
config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth = True
sess=tf.Session(config=config)

# Input placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 30,30,48], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 2], name='y-input')
    keep_prob = tf.placeholder(tf.float32)
    
with tf.variable_scope('model'):
    net = nn()
    #inp = tf.pad(tf.reshape(x, [FLAGS.batch_size,30,30,1]),[[0,0],[2,2],[2,2],[0,0]])
    inp = tf.reshape(x, [FLAGS.batch_size,30,30,48])
    op = net.forward(inp)
    y = tf.squeeze(op)
    trainer = net.fit(output=y,ground_truth=y_,loss='softmax_crossentropy',optimizer='adam', opt_params=[FLAGS.learning_rate])
with tf.variable_scope('relevance'):
    if FLAGS.relevance:
        LRP = net.lrp(op, FLAGS.relevance_method, 1)

        # LRP layerwise 
        relevance_layerwise = []
        R = op
        for layer in net.modules[::-1]:
            R = net.lrp_layerwise(layer, R, FLAGS.relevance_method, 1)
            relevance_layerwise.append(R)

    else:
        LRP=[]
        relevance_layerwise = []
            
with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32))
tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries
merged = tf.summary.merge_all()


init=tf.global_variables_initializer()
sess.run(init)

    
validation_x=list(np.array(validation)[:,0])
validation_y=list(np.array(validation)[:,1])
validation_acc = []
train_acc = []
validation_loss=[]
#saver=tf.train.Saver(max_to_keep=1)


##For trained model
saver=tf.train.Saver()
saver.restore(sess,'E:/revisiondata/train/holiday_2_200.cpk')

##For model training
# start=time.clock()
# for i in range(FLAGS.max_steps):
#     sample=random.sample(train,FLAGS.batch_size)
#     batch_x=list(np.array(sample)[:,0])
#     batch_y=list(np.array(sample)[:,1])
        
#     train_inp = {x:batch_x, y_: batch_y, keep_prob: FLAGS.dropout}
#     summary, _ , train_accuracy, relevance_train,op, rel_layer,train_loss= sess.run([merged, trainer.train,accuracy, LRP,y, relevance_layerwise,trainer.cost], feed_dict=train_inp)
#     if i % FLAGS.test_every==0:
#         accuracys=[]
#         for j in range(int(len(validation)/FLAGS.batch_size)):
#             #sample_validation=random.sample(validation,FLAGS.batch_size)
#             sample_validation=validation[j*FLAGS.batch_size:j*FLAGS.batch_size+FLAGS.batch_size]
#             sv_x=list(np.array(sample_validation)[:,0])
#             sv_y=list(np.array(sample_validation)[:,1])
#             validation_inp = {x:sv_x, y_: sv_y, keep_prob: 1}
#             #validation_inp = {x:validation_x, y_: validation_y, keep_prob: FLAGS.dropout}
#             newaccuracy= sess.run([accuracy], feed_dict=validation_inp)
#             accuracys.append(newaccuracy[0])
#         validation_accuracy=sum(accuracys)/(len(validation)/FLAGS.batch_size)
#         validation_acc.append(validation_accuracy)
#         train_acc.append(train_accuracy)
#         validation_loss.append(train_loss)
#         print(str(i)+','+str(validation_accuracy)+','+str(train_accuracy)+','+str(train_loss))
#         if(float(validation_accuracy)>0.95):
#             break
# endtime=time.clock()
# print(endtime-start)

In [ ]:
##The assessed importance order of each spatial unit

order=[
    429,524,766,321,262,381,383,525,382,320,850,289,351,350,378,377,384,261,408,745,322,374,232,356,231,230,430,409,290,282,806,438,
    790,174,441,459,319,523,744,288,411,373,368,776,432,431,260,113,372,343,323,380,442,258,325,555,197,173,371,775,465,807,667,556,
    492,494,439,463,112,385,312,836,144,466,348,349,342,291,379,554,399,129,204,501,879,557,201,433,365,614,200,170,402,584,326,386,
    254,500,159,553,428,517,529,347,338,471,357,559,101,531,142,199,434,583,229,198,143,367,440,216,224,318,366,522,188,403,175,460,
    222,228,353,195,738,743,404,190,585,493,309,311,248,168,458,217,196,436,369,158,286,558,280,489,283,352,161,496,740,194,528,355,
    108,820,234,798,128,77,162,256,413,462,414,443,317,706,341,252,306,774,805,586,622,504,472,223,316,495,741,138,220,172,284,314,
    191,736,235,395,257,760,425,407,278,437,878,768,287,337,133,310,107,387,400,461,637,405,589,464,490,470,145,703,218,281,82,295,
    410,671,139,227,354,865,804,468,415,250,412,100,259,221,527,336,445,535,324,427,715,899,444,164,455,502,526,294,114,165,473,130,
    487,607,606,406,560,308,189,109,486,615,532,344,52,868,469,435,565,849,538,518,676,566,166,83,491,339,533,249,761,296,134,279,
    416,617,582,335,305,315,424,111,644,396,638,579,388,389,401,131,266,247,837,398,370,548,219,686,894,255,167,313,160,536,78,137,
    763,547,497,364,739,203,426,588,135,581,446,251,132,838,169,205,613,208,193,192,747,829,519,85,666,187,880,576,773,636,253,75,
    163,397,516,503,587,787,505,748,521,293,233,515,99,731,346,580,746,498,882,530,546,835,655,297,791,687,215,537,711,392,620,643,
    851,334,186,860,245,345,76,340,393,105,104,456,71,474,697,57,848,564,866,590,823,72,762,141,650,477,84,895,567,376,737,716,
    881,103,307,269,616,563,277,817,171,226,717,98,375,102,641,358,106,157,275,733,304,292,457,417,394,69,276,74,710,562,97,608,
    611,115,549,734,467,819,646,653,681,764,602,767,86,701,42,202,520,898,618,263,53,593,625,609,652,447,714,597,391,685,246,777,
    803,719,56,821,645,572,771,545,867,897,669,735,239,422,359,854,853,488,797,507,539,81,87,809,642,552,551,592,619,712,578,859,
    639,238,869,264,683,544,893,110,140,327,577,514,772,651,244,756,225,896,484,677,730,361,423,679,66,718,689,786,55,209,709,298,
    623,840,212,27,640,670,44,824,499,534,362,67,883,41,73,303,770,688,136,596,328,759,285,626,864,54,890,612,808,792,668,482,
    360,51,561,485,852,70,889,116,513,742,418,863,543,154,575,769,834,857,185,649,627,79,861,855,732,454,672,483,363,542,267,678,
    624,799,22,870,648,333,704,673,206,707,727,633,660,419,178,237,476,68,684,39,845,621,243,302,876,875,599,265,647,12,390,789,
    758,47,448,48,207,176,449,598,862,682,153,25,156,117,509,690,301,9,796,856,96,726,713,421,43,700,603,452,26,698,757,610,
    5,788,779,568,674,839,273,605,778,822,841,183,858,18,14,45,696,299,794,236,6,708,833,811,211,574,179,659,594,591,630,35,
    844,828,124,37,749,816,123,330,420,595,478,628,146,65,506,59,332,665,34,21,512,658,657,155,765,571,40,702,177,573,19,600,
    151,729,20,604,884,886,90,654,300,268,830,152,801,24,570,49,846,63,270,17,213,7,479,699,720,481,569,818,58,50,793,802,
    118,13,28,242,891,887,127,842,453,826,91,29,46,64,214,827,36,871,847,814,656,150,508,126,635,810,240,785,475,815,271,23,
    274,550,728,885,11,241,750,629,751,62,147,680,755,843,705,210,825,877,892,88,694,675,329,632,888,8,184,180,795,874,800,120,
    725,121,272,634,780,664,122,541,16,10,691,61,80,181,721,784,812,511,450,451,331,15,663,93,510,149,33,148,94,89,119,540,
    92,125,695,781,601,831,724,60,832,480,95,182,754,813,723,753,722,783,782,693,692,38,631,30,872,873,662,4,661,1,2,32,
    3,31,0,752
]
groupnum=[9,24,45,74,111,156,211,282,387]
def getrc(n):
    a=int(n/30)
    b=n%30
    return a,b


In [ ]:
##The volume order of each spatial unit
F=open("E:/revisiondata/holiday_2_total_2016",'rb')
total2016=pickle.load(F)
F.close()
volume=list(np.array(total2016)[:,0])
meanvolume=np.mean(np.array(volume),axis=0)
totalvolume=np.array(np.array(copy.deepcopy(meanvolume)).sum(axis=2)).reshape(900)
volumeorder=np.argsort(-totalvolume)
#print(volumeorder)

In [ ]:
##Validation #1, replaced the values of several zones and get the classification accuracy
repeatnum=50
#for each group
for v in range(len(groupnum)):
    replacenum=groupnum[v]
    totalaccuracy=[]
    #for each experiment
    for repeat in range(repeatnum):
        accuracys=[]
        #for each input sample
        for j in range(len(total2016)):
            sample_test=[]
            tmpimage=copy.deepcopy(total2016[j])
            tmpimage_raw=copy.deepcopy(total2016[j])
        
            hourimg=np.array(tmpimage[0]).reshape(900,48)
            
            baseline=random.sample(order,replacenum)
            
            
            ##interpolation by eight neighbours
            for t in range(replacenum):
                ##different replacement orders
                
                #rnum, cnum=getrc(order[t])
                #rnum, cnum=getrc(volumeorder[t])
                rnum,cnum=getrc(baseline[t])
                
                hourmean=np.zeros(48)
                rvalue=[1,-1,0,0,1,1,-1,-1]
                cvalue=[0,0,1,-1,1,-1,1,-1]
                num=0
                for neigh in range(8):
                    newrnum=rnum+rvalue[neigh]
                    newcnum=cnum+cvalue[neigh]
                    if(newrnum>=0 and newrnum<30 and newcnum>=0 and newcnum<30):
                        hourmean=copy.deepcopy(hourmean)+np.array(copy.deepcopy(tmpimage_raw[0][newrnum][newcnum]))
                        num+=1
                hourmean/=num        
                
                for d in range(48):
                    tmpimage[0][rnum][cnum][d]=hourmean[d] 
        
            for k in range(5):
                sample_test.append(tmpimage)
            st_x=list(np.array(sample_test)[:,0])
            st_y=list(np.array(sample_test)[:,1])
            test_inp = {x:st_x, y_: st_y, keep_prob: 1}
            newaccuracy= sess.run([accuracy], feed_dict=test_inp)
            accuracys.append(newaccuracy[0])
        #for all samples
        test_accuracy=sum(accuracys)/len(total2016)
        totalaccuracy.append(test_accuracy)
    #print(totalaccuracy)
    print(np.mean(totalaccuracy))